<a href="https://colab.research.google.com/github/galaga2015/QnATrainingKnowledge-base/blob/main/calarm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain tiktoken chromadb pypdf transformers InstructorEmbedding
!pip -q install accelerate bitsandbytes sentencepiece Xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.8/269.8 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 113.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 115.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 110.0 M

In [ ]:
!pip show langchain
!pip show transformers

Name: langchain
Version: 0.0.251
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
Name: transformers
Version: 4.31.0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [ ]:
import zipfile
import os

# Mount your Google Drive to Colab
from google.colab import drive
drive.mount('/content/drive')

# Create a ZipFile object with the path to the zip file
zip_file_path = '/content/drive/My Drive/training.zip'
with zipfile.ZipFile(zip_file_path, 'r') as zipObj:

    # Create a directory to store the extracted files
    training_directory = 'training'
    if not os.path.exists(training_directory):
        os.mkdir(training_directory)

    # Use the extractall() method to extract the contents of the zip file to the specified location
    zipObj.extractall(training_directory)

# Delete the zipfile
#os.remove(zip_file_path)


Mounted at /content/drive


In [ ]:
# Get the list of files in the directory
# !mkdir "training"
# !ls -i "/content/drive/My Drive/calarm_training"
# !cp -r "/content/drive/My Drive/calarm_training/." "training"

!mv ./training/Calculate\ Amount\ Due\ Based\ on\ Census\ Data\ for\ Calarm\ \(1\).txt ./training/CalculateDebtFromCensus.txt
!mv ./training/Collect\ Payments\ \(1\).txt ./training/CollectPayments.txt
!mv ./training/Creating\ a\ Payment\ Plan\ and\ approval\ \(1\).txt ./training/CreatePaymentPlan.txt
!ls -i training


mv: cannot stat './training/Calculate Amount Due Based on Census Data for Calarm (1).txt': No such file or directory
mv: cannot stat './training/Collect Payments (1).txt': No such file or directory
mv: cannot stat './training/Creating a Payment Plan and approval (1).txt': No such file or directory
2229517 'Calculate Amount Due Based on Census Data for Calarm.txt'
2229518 'Collect Payments.txt'
2229519 'Creating a Payment Plan and approval.txt'


In [ ]:
import torch
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline

tokenizer = LlamaTokenizer.from_pretrained("TheBloke/wizardLM-7B-HF")
model = LlamaForCausalLM.from_pretrained("TheBloke/wizardLM-7B-HF",
                                                  load_in_8bit=True,
                                                  device_map='auto',
                                                  torch_dtype=torch.float16,
                                                  low_cpu_mem_usage=True
                                                  )

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


KeyboardInterrupt: ignored

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
import torch

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2000,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
print(local_llm('What is the capital of Israel?'))


Israel has no official capital city. Jerusalem is considered to be the holy city for both Jews and Muslims, but it is not recognized as the capital by the international community due to political tensions in the region. Tel Aviv is often referred to as the commercial capital of Israel.


In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=b5d91d29017b4d8d27e98462e025a16560701dd0c1d01fe4bc4ca7b2a23af293
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [ ]:
# Create a DirectoryLoader object
loader = DirectoryLoader("training", glob="*.txt", loader_cls=TextLoader)

# Load the text files
documents = loader.load()


# # Print the names of the loaded documents
# for document in documents:
#     with open(document.path, 'r', encoding='utf-8') as f:
#         print(document.name, f.read())

In [ ]:
len(documents)

3

In [ ]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [ ]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})


load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=local_llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "what are the steps for calculating debt from cencus"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 To calculate debt from census data using CalARM, follow these steps:

Step 1: Create a provider and facility record in CalARM. This will allow you to track the facility's
information and history.

Step 2: Enter the census data into CalARM. This should include the number of resident days for the month, as
well as any other relevant information such as program type, case record type, and status.

Step 3: Review the census data for accuracy and completeness. If there are any errors or omissions, you may
need to resubmit the data or contact the facility to obtain missing information.

Step 4: Once the census data has been reviewed and verified, CalARM will calculate the debt owed by the
facility based on the resident days and other factors.

Step 5: If there are errors or discrepancies in the census data, CalARM will flag the task and create a
request for the facility to resubmit the data or provide additional information.

Step 6: If the census data is accurate and complete, CalARM will 

In [ ]:
query = "how do I enter in the bed days for a facility?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 To enter bed days for a facility in CalARM, you would need to create a new census case for the facility and
input the necessary information such as the facility name, contact person, and start date of the census
period. Once you have created the census case, you can input the total number of resident days for the
facility during the census period. This can be done by either manually entering the data or importing it from
another source. After the data has been entered, CalARM will calculate the amount due based on the
reimbursement rate set by the state.


Sources:
training/Calculate Amount Due Based on Census Data for Calarm.txt
training/Calculate Amount Due Based on Census Data for Calarm.txt
training/Calculate Amount Due Based on Census Data for Calarm.txt
